In [ ]:
%matplotlib inline
from __future__ import print_function
import os
import random
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import ndimage

import improc

In [ ]:
reload (improc)

In [ ]:
# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

# Enumerate Images
Image names are sequential, so add every tenth image to the validation set based on filename.

In [ ]:
training = []
test = []

for root, dirs, files in os.walk('captures'):
    for name in files:
        path = os.path.join(root, name)
        low_name = name.lower()
        # Find all the image files, split into test and training.
        if low_name.endswith(".png"):
            if low_name.endswith("0.png"):
                test.append(path)
            else:
                training.append(path)

print("Training:", len(training), "Test:", len(test))

In [ ]:
training[:5]

In [ ]:
test[:5]

# Image Processing
Each image file contains a color image (top half), and an encoded depth image (bottom half)
<img src="testing/IMG_2114.PNG">
* Note: The image may also contain the orientation data. If so it is encoded in the first two pixels of the depth image. If the first pixel of the depth image is red, the second has the x, y, z, w quaternion components encoded in the r,g,b,a values.

The improc module contains functions for splitting the image, decoding the depth back into floating point millimeters, and for filling in gaps.

In [ ]:
def prepare_image(image_path, strategy=np.mean):
    combined_image = ndimage.imread(image_path)
    color_image, depth_image = improc.split(combined_image)
    depths, attitude = improc.decode_depth(depth_image)
    cleaned = improc.mipmap_imputer(depths, strategy)
    return (color_image, depths, attitude, cleaned)

In [ ]:
prepared_image = prepare_image("testing/IMG_2114.PNG")
plt.imshow(prepared_image[1])

In [ ]:
plt.imshow(prepared_image[3])